In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from time import time 
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import seaborn as sns
%matplotlib inline

In [2]:
# Load training data as dataframe
df = pd.read_csv('final_train.csv')

display(df)

,Unnamed: 0,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),...,tGravityAccMag-energy(),tGravityAccMag-entropy(),tGravityAccMag-iqr(),tGravityAccMag-mad(),tGravityAccMag-max(),tGravityAccMag-mean(),tGravityAccMag-min(),tGravityAccMag-sma(),tGravityAccMag-std(),void()
0,0,STANDING,-1.574191,-0.805700,-2.453831,-1.861714,0.115517,-2.023369,3.125860,-9.474436,...,5.049130,3.272281,5.103018,-0.823302,3.063681,-0.928676,-1.007695,3.094236,1.669987,1.0
1,1,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,-2.187890,-0.544652,2.0
2,2,LAYING,-1.548333,-0.139628,-2.380889,-1.915239,0.067613,-0.887342,6.007361,-11.906904,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.071755,2.024442,3.0
3,3,WALKING,-1.574094,-0.793443,-2.449959,-1.842741,-0.195625,-0.950671,2.104353,-11.906121,...,4.860758,4.177535,3.950012,-0.322642,2.366287,-0.866937,-0.696431,1.863375,0.550414,2.0
4,4,LAYING,-1.548333,-0.139628,-2.380889,NaN,0.067613,-0.887342,6.007361,-11.970485,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.102513,2.024442,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,11081,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,3.534816,-0.544652,2.0
11082,11082,LAYING,-1.550288,-0.231565,-2.376561,-1.868524,0.007653,-0.077020,5.494466,-5.860692,...,5.048737,2.923856,5.182026,-0.998495,2.853755,-0.933561,-1.000173,-1.984575,1.372763,1.0
11083,11083,WALKING_UPSTAIRS,-1.569815,-0.794940,-2.482325,-1.784151,0.021844,-0.740329,8.005261,-11.916107,...,4.453514,4.302852,2.979961,-6.032382,1.853029,-0.802817,0.041975,3.427548,-0.075184,1.0
11084,11084,WALKING_DOWNSTAIRS,-1.574768,-0.752847,-2.449529,-1.737755,-0.189761,-0.454246,8.054443,-5.611277,...,4.615939,4.443277,3.208597,-6.200222,1.675269,-0.828934,-0.877004,3.385405,-0.383217,3.0


In [4]:
# Correct the order of values and label
del df["Unnamed: 0"]

# Calculate correlation matrix between columns
cor_matrix = df.corr().abs()

# Drop highest correlated columns
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
df_analysis = df.drop(df[to_drop], axis=1) 


KeyError: 'Unnamed: 0'

In [ ]:
display(df_analysis)

In [16]:
# Divide dataframe into values and labels
Y_labels = df_analysis['Activity']
X_values = df_analysis.loc[:, df_analysis.columns != 'Activity']

# Save columns names 
variables = list(X_values.columns.values)
display(X_values)

,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),"fBodyAcc-bandsEnergy()-1,16","fBodyAcc-bandsEnergy()-1,16.1",...,tGravityAcc-entropy()-X,tGravityAcc-entropy()-Y,tGravityAcc-entropy()-Z,tGravityAcc-iqr()-X,tGravityAcc-iqr()-Y,tGravityAcc-iqr()-Z,tGravityAcc-sma(),tGravityAccMag-mad(),tGravityAccMag-sma(),void()
0,-1.574191,-0.805700,-2.453831,-1.861714,0.115517,-2.023369,3.125860,-9.474436,3.622600,6.137246,...,8.394366,3.246233,0.937300,4.725389,-1.578871,-0.442374,6.357369,-0.823302,3.094236,1.0
1,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,2.512880,5.740178,...,8.112279,1.926083,2.348895,4.719540,-1.521161,-0.463599,6.077734,-2.689605,-2.187890,2.0
2,-1.548333,-0.139628,-2.380889,-1.915239,0.067613,-0.887342,6.007361,-11.906904,3.621984,6.158630,...,7.690866,3.055395,1.937417,4.684114,-1.580576,-0.359234,5.893237,-0.549843,-2.071755,3.0
3,-1.574094,-0.793443,-2.449959,-1.842741,-0.195625,-0.950671,2.104353,-11.906121,3.355701,5.695567,...,8.394366,3.246233,1.256970,4.708756,-1.557720,-0.407105,6.314269,-0.322642,1.863375,2.0
4,-1.548333,-0.139628,-2.380889,NaN,0.067613,-0.887342,6.007361,-11.970485,3.621984,6.158630,...,7.690866,3.055395,1.937417,4.684114,-1.580576,-0.359234,5.893237,-0.549843,-2.102513,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,2.512880,5.740178,...,8.112279,1.926083,2.348895,4.719540,-1.521161,-0.463599,6.077734,-2.689605,3.534816,2.0
11082,-1.550288,-0.231565,-2.376561,-1.868524,0.007653,-0.077020,5.494466,-5.860692,3.603169,6.137144,...,8.093967,2.715915,1.913262,4.724055,-1.576766,-0.375922,5.869809,-0.998495,-1.984575,1.0
11083,-1.569815,-0.794940,-2.482325,-1.784151,0.021844,-0.740329,8.005261,-11.916107,2.644861,5.436273,...,7.189854,3.246233,2.348895,4.499738,-1.464568,-0.856437,6.057005,-6.032382,3.427548,1.0
11084,-1.574768,-0.752847,-2.449529,-1.737755,-0.189761,-0.454246,8.054443,-5.611277,2.473134,5.433737,...,8.070087,3.246233,0.239752,4.633185,-1.404707,-0.637358,6.379030,-6.200222,3.385405,3.0


In [6]:
# Standardising the data
# Center to the mean and component wise scale to unit variance
# for col in [variables]:
#     X_all[col] = scale(X_all[col])
sc = StandardScaler()
X_values = sc.fit_transform(X_values)

In [7]:
# Dividing data into training and test set taking 80% of prior data as a training set,
# so that our models are tested based on matches that took place after matches from our training set
X_train, X_test= np.split(X_values, [int(.8 *len(X_values))])
Y_train, Y_test= np.split(Y_labels, [int(.8 *len(Y_labels))])

In [8]:
# Printing shape of our data sets
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (8868, 157)
Training Labels Shape: (8868,)
Testing Features Shape: (2218, 157)
Testing Labels Shape: (2218,)


In [9]:
# Training the classifier and measuring its time
def train(clf, X_train, Y_train):
    
    # Starting the clock, training the classifier, then stoping the clock
    start = time()
    clf.fit(X_train, Y_train)
    end = time()
    
    # Printing the results
    print ("Model trained in: ",end - start)

In [18]:
# Making predictions 
def predict(clf, features, target):
    
    # Starting the clock, making predictions, then stoping the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Printing results
    print ("Prediction made in: ",end - start)
    
    # Returning F1 score and accuracy
    return f1_score(target, y_pred, average=None), sum(target == y_pred) / float(len(y_pred))

In [11]:
# Evaluation of our model
def fit(clf, X_train, y_train, X_test, y_test):
    
    # Indicating the model and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Training model
    train(clf, X_train, y_train)
    
    # Printing the results of prediction for both training and testing set
    f1, acc = predict(clf, X_train, y_train)
    print ("===========================================")
    print ("Indexes assesing methods for training set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)
    
    print ("===========================================")
    f1, acc = predict(clf, X_test, y_test)
    print ("Indexes assesing methods for test set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)

In [12]:
# Inference function
def inference(clf_XGB, inf):
    start = time()
    y_pred = clf_XGB.predict(inf)
    end = time()

    # Printing results
    print ("Inference made in: ",end - start)

In [19]:
# Creating our model and training it
clf_XGB = xgb.XGBClassifier()

fit(clf_XGB, X_train, Y_train, X_test, Y_test)
print ('')

Training a XGBClassifier using a training set size of 8868. . .
[03:50:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Model trained in:  27.987999439239502
Prediction made in:  0.046999216079711914
Indexes assesing methods for training set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1. 1. 1. 1. 1. 1.]
Accuracy: 1.0
Prediction made in:  0.014999151229858398
Indexes assesing methods for test set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1.         0.98701299 0.9867374  0.99337748 0.98686679 0.99298738]
Accuracy: 0.9914337240757439



In [ ]:
# Inference time
pred_try = np.array([X_train[0]])
inference(clf_XGB, pred_try)

In [ ]:
# Saving trained model
clf_XGB.save_model('final.model')

In [13]:
# Loading trained model
clf2 = xgb.XGBClassifier()
clf2.load_model('final.model')

In [14]:
f1f, accf, predicted = predict(clf2, X_values, Y_labels)

Prediction made in:  0.12299656867980957


In [15]:
np.mean(predicted == df.Activity)

0.9982861266462204